# SWELL-KW FastGRNN

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import datetime as datetime
import pickle as pkl
from sklearn.model_selection import train_test_split
import pathlib
from os import mkdir

In [2]:
def loadData(dirname):
    x_train = np.load(dirname + '/' + 'x_train.npy')
    y_train = np.load(dirname + '/' + 'y_train.npy')
    x_test = np.load(dirname + '/' + 'x_test.npy')
    y_test = np.load(dirname + '/' + 'y_test.npy')
    x_val = np.load(dirname + '/' + 'x_val.npy')
    y_val = np.load(dirname + '/' + 'y_val.npy')
    return x_train, y_train, x_test, y_test, x_val, y_val
def makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir):
    x_train, y_train, x_test, y_test, x_val, y_val = loadData(sourceDir)
    x, y = bagData(x_train, y_train, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_train.npy', x)
    np.save(outDir + '/y_train.npy', y)
    print('Num train %d' % len(x))
    x, y = bagData(x_test, y_test, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_test.npy', x)
    np.save(outDir + '/y_test.npy', y)
    print('Num test %d' % len(x))
    x, y = bagData(x_val, y_val, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_val.npy', x)
    np.save(outDir + '/y_val.npy', y)
    print('Num val %d' % len(x))
def bagData(X, Y, subinstanceLen, subinstanceStride):
    numClass = 2
    numSteps = 20
    numFeats = 22
    assert X.ndim == 3
    print(X.shape)
    assert X.shape[1] == numSteps
    assert X.shape[2] == numFeats
    assert subinstanceLen <= numSteps
    assert subinstanceLen > 0
    assert subinstanceStride <= numSteps
    assert subinstanceStride >= 0
    assert len(X) == len(Y)
    assert Y.ndim == 2
    assert Y.shape[1] == numClass
    x_bagged = []
    y_bagged = []
    for i, point in enumerate(X[:, :, :]):
        instanceList = []
        start = 0
        end = subinstanceLen
        while True:
            x = point[start:end, :]
            if len(x) < subinstanceLen:
                x_ = np.zeros([subinstanceLen, x.shape[1]])
                x_[:len(x), :] = x[:, :]
                x = x_
            instanceList.append(x)
            if end >= numSteps:
                break
            start += subinstanceStride
            end += subinstanceStride
        bag = np.array(instanceList)
        numSubinstance = bag.shape[0]
        label = Y[i]
        label = np.argmax(label)
        labelBag = np.zeros([numSubinstance, numClass])
        labelBag[:, label] = 1
        x_bagged.append(bag)
        label = np.array(labelBag)
        y_bagged.append(label)
    return np.array(x_bagged), np.array(y_bagged)

In [3]:
subinstanceLen=8
subinstanceStride=3
extractedDir = '/home/sf/data/SWELL-KW/'
#mkdir('/home/sf/data/SWELL-KW/FG_8_3')
rawDir = extractedDir + '/RAW'
sourceDir = rawDir
outDir = extractedDir + '/FG_%d_%d/' % (subinstanceLen, subinstanceStride)
makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir)

(3679, 20, 22)
Num train 3679
(1022, 20, 22)
Num test 1022
(409, 20, 22)
Num val 409


In [4]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

# FastGRNN and FastRNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_FastGRNN
from edgeml.graph.rnn import EMI_FastRNN
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

In [5]:
# Network parameters for our FastGRNN + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 8
NUM_FEATS = 22
FORGET_BIAS = 1.0
NUM_OUTPUT = 2
USE_DROPOUT = False
KEEP_PROB = 0.9

# Non-linearities can be chosen among "tanh, sigmoid, relu, quantTanh, quantSigm"
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"

# Ranks of Parameter matrices for low-rank parameterisation to compress models.
WRANK = 5
URANK = 6

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 3
# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4
# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 30

# A staging direcory to store models
MODEL_PREFIX = '/home/sf/data/SWELL-KW/FG_8_13/model-fgrnn'

# Loading Data

In [6]:
# Loading the data
path='/home/sf/data/SWELL-KW/FG_8_3/'
x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (3679, 5, 8, 22)
y_train shape is: (3679, 5, 2)
x_test shape is: (409, 5, 8, 22)
y_test shape is: (409, 5, 2)


# Computation Graph

In [7]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
    if inference is False:
        feedDict = {self._emiGraph.keep_prob: keep_prob}
    else:
        feedDict = {self._emiGraph.keep_prob: 1.0}
    return feedDict

    
EMI_FastGRNN._createExtendedGraph = createExtendedGraph
EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
if USE_DROPOUT is True:
    EMI_FastGRNN.feedDictFunc = feedDictFunc

In [8]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiFastGRNN = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                           gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

In [9]:
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (3679, 5, 8, 22)
y_train shape is: (3679, 5, 2)
x_test shape is: (409, 5, 8, 22)
y_test shape is: (409, 5, 2)


In [10]:
tf.reset_default_graph()
g1 = tf.Graph()    
with g1.as_default():
    # Obtain the iterators to each batch of the data
    x_batch, y_batch = inputPipeline()
    # Create the forward computation graph based on the iterators
    y_cap = emiFastGRNN(x_batch)
    # Create loss graphs and training routines
    emiTrainer(y_cap, y_batch)

# EMI Driver 

In [11]:
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiFastGRNN, emiTrainer)

emiDriver.initializeSession(g1)
y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                      y_train=y_train, bag_train=BAG_TRAIN,
                                      x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                      numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                      numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                      numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                      fracEMI=0.5, updatePolicy='top-k', k=1)

Update policy: top-k
Training with MI-RNN loss for 15 rounds
Round: 0
Epoch   2 Batch   100 (  330) Loss 0.09010 Acc 0.49375 | Val acc 0.65037 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1000
Epoch   2 Batch   100 (  330) Loss 0.08737 Acc 0.53750 | Val acc 0.66748 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1001
Epoch   2 Batch   100 (  330) Loss 0.08466 Acc 0.50625 | Val acc 0.68215 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1002
Epoch   2 Batch   100 (  330) Loss 0.08016 Acc 0.57500 | Val acc 0.69193 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1003
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1003
Round: 1
Epoch   2 Batch   100 (  330) Loss 0.07575 Acc 0.64375 | Val acc 0.70416 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1004
Epoch   2 Batch   100 (  330) Loss 0.07176 Acc 0.71875 | Val acc 0.71883 | M

Epoch   2 Batch   100 (  330) Loss 0.03206 Acc 0.88750 | Val acc 0.83863 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1047
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1046
Round: 12
Epoch   2 Batch   100 (  330) Loss 0.03206 Acc 0.88750 | Val acc 0.83863 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1048
Epoch   2 Batch   100 (  330) Loss 0.03282 Acc 0.89375 | Val acc 0.83863 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1049
Epoch   2 Batch   100 (  330) Loss 0.03429 Acc 0.89375 | Val acc 0.83619 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1050
Epoch   2 Batch   100 (  330) Loss 0.03082 Acc 0.92500 | Val acc 0.84108 | Model saved to /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn, global_step 1051
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1051
Round: 13
Epoch   2 Batch   100 (  330) Loss 0.02883 Ac

# Evaluating the  trained model

In [12]:
# Early Prediction Policy: We make an early prediction based on the predicted classes
#     probability. If the predicted class probability > minProb at some step, we make
#     a prediction at that step.
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
    predictionStep = predictionStep + 1
    predictionStep = np.reshape(predictionStep, -1)
    totalSteps = np.sum(predictionStep)
    maxSteps = len(predictionStep) * numTimeSteps
    savings = 1.0 - (totalSteps / maxSteps)
    if returnTotal:
        return savings, totalSteps
    return savings

In [13]:
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb, minProb=0.99)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS))

Accuracy at k = 2: 0.868885
Additional savings: 0.295475


In [14]:
# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.860078   0.859642   0.868740   0.862055   0.860078   0.860078   
1    2  0.868885   0.868860   0.870591   0.869796   0.868885   0.868885   
2    3  0.875734   0.875619   0.875752   0.875529   0.875734   0.875734   
3    4  0.866928   0.866436   0.869102   0.865945   0.866928   0.866928   
4    5  0.863992   0.862807   0.871518   0.862185   0.863992   0.863992   

   micro-rec  fscore_01  
0   0.860078   0.867470  
1   0.868885   0.870656  
2   0.875734   0.871847  
3   0.866928   0.858333  
4   0.863992   0.850054  
Max accuracy 0.875734 at subsequencelength 3
Max micro-f 0.875734 at subsequencelength 3
Micro-precision 0.875734 at subsequencelength 3
Micro-recall 0.875734 at subsequencelength 3
Max macro-f 0.875619 at subsequencelength 3
macro-precision 0.875752 at subsequencelength 3
macro-recall 0.875529 at subsequencelength 3


## Picking the best model

In [21]:
k=3
emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX , 1032)
devnull = open(os.devnull, 'r')
for val in modelStats:
    round_, acc, modelPrefix, globalStep = val
    emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
 
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
    print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
    print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS)) 

INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1032
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1003
Round:  0, Validation accuracy: 0.6919, Test Accuracy (k = 3): 0.689824, Additional savings: 0.000024
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1007
Round:  1, Validation accuracy: 0.7237, Test Accuracy (k = 3): 0.708415, Additional savings: 0.000171
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1011
Round:  2, Validation accuracy: 0.7653, Test Accuracy (k = 3): 0.727006, Additional savings: 0.001492
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1014
Round:  3, Validation accuracy: 0.7775, Test Accuracy (k = 3): 0.732877, Additional savings: 0.002275
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1019
Round:  4, Validation accuracy: 0.7848, Test Accurac

In [22]:
MODEL_PREFIX = '/home/sf/data/SWELL-KW/FG_8_13/model-fgrnn'

In [23]:
import time
k=2
start = time.time()
emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX , 1118)
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)###
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)###
end = time.time()
print(start-end)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/FG_8_13/model-fgrnn-1118
-1.243401288986206
Accuracy at k = 2: 0.870841


In [18]:
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 8, #subinstance length.
    "NUM_FEATS" : 22,
    "FORGET_BIAS" : 1.0,
    "UPDATE_NL" : "quantTanh",
    "GATE_NL" : "quantSigm",
    "NUM_OUTPUT" : 3,
    "WRANK" : 5,
    "URANK" : 6,
    "USE_DROPOUT" : False,
    "KEEP_PROB" : 0.9,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 2,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
    "MODEL_PREFIX" : '/home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn'
}

In [24]:
fgrnn_dict = {**params}
fgrnn_dict["k"] = k
fgrnn_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
fgrnn_dict["total_savings"] = getEarlySaving(predictionStep, NUM_TIMESTEPS)
fgrnn_dict["y_test"] = BAG_TEST
fgrnn_dict["y_pred"] = bagPredictions

# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
print (tabulate(df, headers=list(df.columns), tablefmt='grid'))

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.853229   0.852651   0.863397   0.855376   0.853229   0.853229   
1    2  0.870841   0.870792   0.873219   0.871904   0.870841   0.870841   
2    3  0.878669   0.878602   0.878571   0.878641   0.878669   0.878669   
3    4  0.868885   0.868431   0.870852   0.867953   0.868885   0.868885   
4    5  0.869863   0.868957   0.875526   0.868309   0.869863   0.869863   

   micro-rec  fscore_01  
0   0.853229   0.861878  
1   0.870841   0.873321  
2   0.878669   0.875752  
3   0.868885   0.860707  
4   0.869863   0.858058  
Max accuracy 0.878669 at subsequencelength 3
Max micro-f 0.878669 at subsequencelength 3
Micro-precision 0.878669 at subsequencelength 3
Micro-recall 0.878669 at subsequencelength 3
Max macro-f 0.878602 at subsequencelength 3
macro-precision 0.878571 at subsequencelength 3
macro-recall 0.878641 at subsequencelength 3
+----+-------+----------+-------------+-------------+-------------+-------

In [25]:
dirname = "/home/sf/data/SWELL-KW/"
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(fgrnn_dict,open(dirname  + filename + ".pkl",mode='wb'))

Results for this run have been saved at /home/sf/data/SWELL-KW/ .
